## Scraping & Data Preparation - Sreality.cz

In [1]:
#import packages
import pandas as pd                    
from pandas import DataFrame            
import numpy as np                     
import matplotlib as plt                
from matplotlib.pyplot import figure    
from collections import Counter         
from datetime import datetime         
import re                               
from time import sleep                 
import random                        
import math                            
import time                           
import itertools                        
import requests                        
from bs4 import BeautifulSoup           
from selenium import webdriver          
import json                             
from geopy.geocoders import Nominatim  
from geopy.exc import GeocoderTimedOut  
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
import os
from IPython.display import Image
import seaborn as sns                
import sys                             

In [2]:
print(requests.get("https://www.sreality.cz/robots.txt").text)

User-agent: *
Disallow: /

User-agent: Googlebot
Allow: /
Disallow: /advertpdf/
Disallow: /favourites-info
Disallow: *_buri=
Disallow: /adresar/*page=
Disallow: /adresar/*perPage=
Disallow: /adresar/*search=
Disallow: /adresar/*letter=
Disallow: /adresar/*id=
Disallow: /firma/*page=
Disallow: /firma/*perPage=
Disallow: /firma/*search=
Disallow: /firma/*letter=
Disallow: /firma/*id=
Disallow: /hledani/*,
Allow: /hledani/*region=*,
Disallow: /rk-detail
Disallow: *bez-aukce=
Disallow: *without-auction=
Disallow: *pois_in_place=
Disallow: *pois_in_place_distance=

User-agent: SeznamBot
Allow: /
Disallow: /advertpdf/
Disallow: /en/
Disallow: /ru/
Disallow: /favourites-info
Disallow: *_buri=
Disallow: /adresar/*page=
Disallow: /adresar/*perPage=
Disallow: /adresar/*search=
Disallow: /adresar/*letter=
Disallow: /adresar/*id=
Disallow: /firma/*page=
Disallow: /firma/*perPage=
Disallow: /firma/*search=
Disallow: /firma/*letter=
Disallow: /firma/*id=
Disallow: /hledani/*,
Allow: /hledani/*region

In [3]:
#Scraping
def get_soup_elements(typ_obchodu = "prodej", typ_stavby = "byty",  lokalita = "praha", pages = 1):  
    
    browser = webdriver.Chrome()
    
    url_x = r"https://www.sreality.cz/hledani"             
    url = url_x + "/" +  typ_obchodu + "/" +  typ_stavby + "/" +  lokalita

    browser.get(url)    # (url).text ??
    sleep(random.uniform(1.0, 1.5))
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,'lxml') # "parser" ??
    
    elements = []    
    
    for link in soup.findAll('a', attrs={'href': re.compile("^/detail/")}):  
        link = link.get('href')   
        elements.append(link)     
    elements = elements[0::2]   

    records = soup.find_all(class_ ='numero ng-binding')[1].text
    records = re.split(r'\D', str(records))                         
    records = ",".join(records).replace(",", "")
    records = int(records)
    max_page = math.ceil(records / 20)   
    print("----------------")
    print("Scrapuji: " + str(typ_obchodu) + " " + str(typ_stavby))
    print("Celkem inzerátů: " + str(records))
    print("Celkem stránek: " + str(max_page))
    
    if pages == 999:    
        pages = max_page
    
    print("Scrapuji (pouze) " + str(pages) + " stran.")
    print("----------------")
    

    
    for i in range(pages-1):   
        i = i+2
        
        sys.stdout.write('\r'+ "Strana " + str(i-1) + " = " + str(round(100*(i-1)/(pages), 2)) + "% progress. Zbývá cca: " + str(round(random.uniform(3.4, 3.8)*(pages-(i-1)), 2 )) + " sekund.")    # Asi upravím čas, na rychlejším kabelu v obýváku je to občas i tak 3 sec :O

        url2 = url + "?strana=" + str(i)
        browser.get(url2)

        sleep(random.uniform(1.0, 1.5))

        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup2 = BeautifulSoup(innerHTML,'lxml') 
        
        elements2 = []
        
        for link in soup2.findAll('a', attrs={'href': re.compile("^/detail/prodej/")}):  
            link = link.get('href') 
            elements2.append(link)  
   
        elements2 = elements2[0::2]  
        
        elements = elements + elements2    

    
    browser.quit()   
    
    return elements

In [4]:
#Getting URLs

def elements_and_ids(x):
    
    elements = pd.DataFrame({"url":x})

    def get_id(x):
        x = x.split("/")[-1]
        return x

    elements["url_id"] = elements["url"].apply(get_id)
    len1 = len(elements)
    elements = elements.drop_duplicates(subset = [ "url", "url_id"], keep = "first", inplace = False)   
    len2 = len(elements)                                                                                                                                                                    
    print("-- Deleted " + str(len1-len2) + " records due to duplication.")
    return elements


In [5]:
#Coordinates, description, price
def get_coords_price_meters(x):
    
    url = "https://www.sreality.cz/api/cs/v2/estates/" + str(x)
    
    response = requests.get(url)
    byt = json.loads(response.text, encoding = 'UTF-8')                                
    try:
        coords = (byt["map"]["lat"], byt["map"]["lon"])
    except:
        coords = (0.01, 0.01) 
    try:
        price = byt["price_czk"]["value_raw"] 
    except:
        price = -1
        
    try:
        description = byt["meta_description"]
    except:
        description = -1
    
    return coords, price, description

def latitude(x):               
    x = str(x).split()[0][1:8]
    return x

def longitude(x):
    x = str(x).split()[1][0:7]
    return x


In [6]:
#Obtaining remaning info
def characteristics(x):
    x = x.split("/")
    buy_rent = x[2]
    home_house = x[3]
    rooms = x[4]
    
    return buy_rent, home_house, rooms

def plocha(x):
    try:
        metry = re.search(r'\s[12]\s\d{3}\s[m]', x)[0] 
        metry = metry.split()[0] + metry.split()[1]     
    except:
        try:
            metry = re.search(r'\s\d{2,3}\s[m]', x)[0] 
            metry = metry.split()[0]                   
        except:
            metry = -1
    return metry

In [7]:
def short_coords(x):
    """
    x = x.astype(str)   
    """
    
    x1 = re.split(r'\W+', x)[1] + "."+re.split(r'\W+', x)[2]
    x1 = round(float(x1), 4)

    x2 = re.split(r'\W+', x)[3] + "."+re.split(r'\W+', x)[4]
    x2 = round(float(x2), 4)

    return (x1, x2)

def adress_old(x):  

    adresy = pd.read_excel("Adresy.xlsx")
    adresy = adresy[["oblast", "město", "okres", "kraj", "url_id", "short_coords"]]
    
    x.short_coords = x.short_coords.astype(str)                             
    data = pd.merge(x, adresy, on=["short_coords", "url_id"], how = "left")  
                                                                         
                                                                          
            
    print("-- Počet doplněných řádků je: " + str(len(data[~data.kraj.isna()])) + ", počet chybějících řádků je: "   + str(len(data[data.kraj.isna()])))
    
    return data

In [8]:
#Adress
def adress_new(x):
  
    geolocator = Nominatim(timeout = 20, user_agent = "JZ_Sreality")   # Pomohlo změnit jméno, proti "Error 403" !!        
    location = geolocator.reverse(x.strip("())"))   
                                                    # Reverse samotné znamená obrácené vyhleádvání = souřadnice -> Adresa
    try:
        oblast  = location[0].split(",")[-7]
    except:
        oblast  = -1
    try:
        město = location[0].split(",")[-6]
    except:
        město  = -1
    try:
        okres = location[0].split(",")[-5]
    except:
        okres  = -1
    try:
        kraj = location[0].split(",")[-4]  
    except:
        kraj  = -1       
    
    time.sleep(0.5)
    return oblast, město, okres, kraj


def repeat_adress(x):
    try:
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))
    except GeocoderTimedOut:
        print("Another try")
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))


def adress_merging(x):

    data_new = x.copy()          
    bool_series = pd.isnull(data_new.kraj)                                   
    data_new = data_new[bool_series]     #subset s chybějícími adresami   
        
    repeat_adress(data_new)
        
    data_all = pd.concat([x, data_new], join_axes=[x.columns])   
    data_all = data_all[~pd.isnull(data_all.kraj)]
    data_all = data_all.sort_index()

    data = data_all.copy()
    
    return data


In [9]:
#Scrap_all
def scrap_all(typ_obchodu = "prodej", typ_stavby = "byty", lokalita= "praha", pages = 1):
    
    data = get_soup_elements(typ_obchodu = typ_obchodu, typ_stavby = typ_stavby, lokalita = lokalita, pages = pages)
    print( "1/8 Data scrapnuta, získávám URLs.")
    
    data = elements_and_ids(data)
    data.to_excel(r"a1_URLs_prodej_byty.xlsx")
    print( "2/8 Získány URL, nyní získávám Souřadnice, Ceny a Popis - několik minut...")
    
    data["coords"], data["cena"], data["popis"] = zip(*data['url_id'].map(get_coords_price_meters))
    data["lat"] = data["coords"].apply(latitude)
    data["lon"] = data["coords"].apply(longitude)
    data.to_excel(r"a2_Souřadnice_prodej_byty.xlsx")
    print( "3/8 Získány Souřadnice, Ceny a Popis, nyní získávám informace z URLs.")
   
    data["prodej"], data["dům"],  data["pokoje"] = zip(*data['url'].map(characteristics))
    print("4/8 Získány informace z URLs, nyní získávám informace z popisu.")
    
    data["plocha"] = data['popis'].apply(plocha)
    data.to_excel(r"a3_Popisky_prodej_byty.xlsx")
    print( "5/8 Získány informace z Popisu, nyní mapuji Adresy z předešlých inzerátů.")
    
    data = pd.read_excel(r"a3_Popisky_prodej_byty.xlsx") 
    data["short_coords"] = data["coords"].apply(short_coords)
    data_upd = adress_old(data)                               
    data = data_upd.copy()
    print( "6/8 Namapovány Adresy z předešlých inzerátů, nyní stahuji nové Adresy - několik minut...")  

    try:                                   
        data_upd = adress_merging(data)   
        data = data_upd.copy()
        data.to_excel(r"a4_SCRAPED_prodej_byty.xlsx")
        print("7+8/8 Získány nové adresy + mergnuto dohromady. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 1.")
    
    except:
        data.to_excel(r"a4_SCRAPED_prodej_byty.xlsx")
        print("7+8/8 ŽÁDNÉ nové adresy. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 1.")
    

    return data

In [ ]:
data = scrap_all(pages=237)

In [11]:
data.head()

,Unnamed: 0,url,url_id,coords,cena,popis,lat,lon,prodej,dům,pokoje,plocha,short_coords,oblast,město,okres,kraj
0,0,/detail/prodej/byt/1+kk/praha-hlubocepy-hugo-h...,1444990812,"(50.0290145874023, 14.3622779846191)",4274096,"Byt 1+kk 43 m² k prodeji Hugo Haase, Praha 5 -...",50.0290,14.3622,prodej,byt,1+kk,43,"(50.029, 14.3623)",NaN,NaN,NaN,NaN
1,1,/detail/prodej/byt/3+kk/praha-nusle-na-veseli/...,2522365020,"(50.0560355285, 14.441434973)",7990000,"Byt 3+kk 75 m² k prodeji Na Veselí, Praha 4 - ...",50.0560,14.4414,prodej,byt,3+kk,75,"(50.056, 14.4414)",NaN,NaN,NaN,NaN
2,2,/detail/prodej/byt/2+1/praha-bubenec-roosevelt...,1044446812,"(50.1036177596022, 14.3994975388416)",8900000,"Byt 2+1 71 m² k prodeji Rooseveltova, Praha 6 ...",50.1036,14.3994,prodej,byt,2+1,71,"(50.1036, 14.3995)",NaN,NaN,NaN,NaN
3,3,/detail/prodej/byt/1+kk/praha-stresovice-stres...,1108790364,"(50.0930545667917, 14.3773498641539)",3299000,"Byt 1+kk 27 m² k prodeji Střešovická, Praha 6 ...",50.0930,14.3773,prodej,byt,1+kk,27,"(50.0931, 14.3773)",NaN,NaN,NaN,NaN
4,4,/detail/prodej/byt/4+kk/praha-nusle-5--kvetna/...,3206889308,"(50.0610050946316, 14.4318384928963)",9490000,"Byt 4+kk 100 m² k prodeji 5. května, Praha 4 -...",50.0610,14.4318,prodej,byt,4+kk,100,"(50.061, 14.4318)",NaN,NaN,NaN,NaN


In [12]:
data.shape

(4708, 17)

In [14]:
data.to_csv('prague_house_prices.csv', index=False)

In [15]:
#data.drop(['AREA', 'REG', 'DEV', 'Type', 'Coverage'], axis=1, inplace=True)
data.columns

Index(['Unnamed: 0', 'url', 'url_id', 'coords', 'cena', 'popis', 'lat', 'lon',
       'prodej', 'dům', 'pokoje', 'plocha', 'short_coords', 'oblast', 'město',
       'okres', 'kraj'],
      dtype='object')

In [ ]:
data.drop(['oblast', 'město', 'okres', 'kraj', 'Unnamed: 0','url','url_id','prodej','dům','coords'], axis=1, inplace=True)

In [20]:
data.drop(['coords'], axis=1, inplace=True)

In [21]:
data.head()

,cena,popis,lat,lon,pokoje,plocha,short_coords
0,4274096,"Byt 1+kk 43 m² k prodeji Hugo Haase, Praha 5 -...",50.0290,14.3622,1+kk,43,"(50.029, 14.3623)"
1,7990000,"Byt 3+kk 75 m² k prodeji Na Veselí, Praha 4 - ...",50.0560,14.4414,3+kk,75,"(50.056, 14.4414)"
2,8900000,"Byt 2+1 71 m² k prodeji Rooseveltova, Praha 6 ...",50.1036,14.3994,2+1,71,"(50.1036, 14.3995)"
3,3299000,"Byt 1+kk 27 m² k prodeji Střešovická, Praha 6 ...",50.0930,14.3773,1+kk,27,"(50.0931, 14.3773)"
4,9490000,"Byt 4+kk 100 m² k prodeji 5. května, Praha 4 -...",50.0610,14.4318,4+kk,100,"(50.061, 14.4318)"


In [22]:
#Make column names meaningful
data.rename(columns={'cena':"Price(CZK)",'popis':'Title','lat':'Latitude','lon':'Longitude','plocha':'Area','short_coords':'Coordinates'}, inplace=True)

In [33]:
data.tail(80)

,Price(CZK),Title,Latitude,Longitude,pokoje,Area,Coordinates
4628,8490000,"Byt 2+kk 68 m² k prodeji Balbínova, Praha 2 - ...",50.0773,14.4335,2+kk,68,"(50.0774, 14.4336)"
4629,4610000,"Byt 1+kk 45 m² k prodeji Jankovcova, Praha 7 -...",50.1105,14.4506,1+kk,45,"(50.1105, 14.4506)"
4630,15751800,"Byt 4+kk 119 m² k prodeji Grafická, Praha 5 - ...",50.0730,14.3965,4+kk,119,"(50.073, 14.3966)"
4631,5900000,"Byt 3+kk 69 m² k prodeji Kandertova, Praha - L...",50.1117,14.4694,3+kk,69,"(50.1117, 14.4694)"
4632,9753000,"Byt 3+kk 76 m² k prodeji Mezilehlá, Praha 9 - ...",50.0949,14.5089,3+kk,76,"(50.095, 14.5089)"
...,...,...,...,...,...,...,...
4703,-1,"Byt 2+kk 47 m² k prodeji Plzeňská, Praha 5 - S...",50.0725,14.3942,2+kk,47,"(50.0725, 14.3942)"
4704,4290000,"Byt 1+kk 35 m² k prodeji Olšanská, Praha 3 - Ž...",50.0833,14.4662,1+kk,35,"(50.0834, 14.4662)"
4705,21679000,"Byt 4+kk 140 m² k prodeji Na pomezí, Praha 5 -...",50.0614,14.3663,4+kk,140,"(50.0615, 14.3663)"
4706,5990000,"Byt 1+kk 39 m² k prodeji Grafická, Praha 5 - S...",50.0730,14.3945,1+kk,39,"(50.0731, 14.3945)"


In [31]:
data.drop(['coords'], axis=1, inplace=True)

,Price(CZK),Area
Price(CZK),1.000000,0.585728
Area,0.585728,1.000000


In [40]:
data_v1=data[data['Price(CZK)']!=-1]

In [41]:
data_v1.shape

(4390, 7)

In [52]:
data_v1.dtypes

Price(CZK)      int64
Title          object
Latitude       object
Longitude      object
pokoje         object
Area            int64
Coordinates    object
dtype: object

In [53]:
data_v1.head()

,Price(CZK),Title,Latitude,Longitude,pokoje,Area,Coordinates
0,4274096,"Byt 1+kk 43 m² k prodeji Hugo Haase, Praha 5 -...",50.0290,14.3622,1+kk,43,"(50.029, 14.3623)"
1,7990000,"Byt 3+kk 75 m² k prodeji Na Veselí, Praha 4 - ...",50.0560,14.4414,3+kk,75,"(50.056, 14.4414)"
2,8900000,"Byt 2+1 71 m² k prodeji Rooseveltova, Praha 6 ...",50.1036,14.3994,2+1,71,"(50.1036, 14.3995)"
3,3299000,"Byt 1+kk 27 m² k prodeji Střešovická, Praha 6 ...",50.0930,14.3773,1+kk,27,"(50.0931, 14.3773)"
4,9490000,"Byt 4+kk 100 m² k prodeji 5. května, Praha 4 -...",50.0610,14.4318,4+kk,100,"(50.061, 14.4318)"


In [ ]:
data_v1.rename(columns={'pokoje':'Type'}, inplace=True)

In [58]:
data_v1.head()

In [77]:
data_v2=data_v1.Title.str.split(",", expand=True)
data_v2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Byt 1+kk 43 m² k prodeji Hugo Haase,Praha 5 - Hlubočepy; 4 274 096 Kč (Uvedená ce...,parkovací stání a případně komoru,sklep.),balkón,výtah,cihlová stavba,osobní vlastnictví,novostavby.,None,None,None,None,None,None,None
1,Byt 3+kk 75 m² k prodeji Na Veselí,Praha 4 - Nusle; 7 990 000 Kč (včetně poplatků,včetně provize,včetně právního servisu),balkón,výtah,cihlová stavba,osobní vlastnictví,ve velmi dobrém stavu.,None,None,None,None,None,None,None
2,Byt 2+1 71 m² k prodeji Rooseveltova,Praha 6 - Bubeneč; 8 900 000 Kč,výtah,cihlová stavba,osobní vlastnictví,ve velmi dobrém stavu.,None,None,None,None,None,None,None,None,None,None
3,Byt 1+kk 27 m² k prodeji Střešovická,Praha 6 - Střešovice; 3 299 000 Kč,cihlová stavba,osobní vlastnictví,ve velmi dobrém stavu.,None,None,None,None,None,None,None,None,None,None,None
4,Byt 4+kk 100 m² k prodeji 5. května,Praha 4 - Nusle; 9 490 000 Kč,terasa,výtah,cihlová stavba,osobní vlastnictví,ve velmi dobrém stavu.,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4699,Byt 2+kk 58 m² k prodeji Tlumačovská,Praha 5 - Stodůlky; 5 850 000 Kč,parkovací místo,výtah,skeletová stavba,osobní vlastnictví,ve velmi dobrém stavu.,None,None,None,None,None,None,None,None,None
4702,Byt 4+kk 126 m² k prodeji Praha 5; 9 989 000 K...,9,989mil.Kč včetně DPH za B2),balkón,terasa,garáž,cihlová stavba,osobní vlastnictví,novostavby.,None,None,None,None,None,None,None
4704,Byt 1+kk 35 m² k prodeji Olšanská,Praha 3 - Žižkov; 4 290 000 Kč,výtah,smíšená stavba,osobní vlastnictví,novostavby.,None,None,None,None,None,None,None,None,None,None
4705,Byt 4+kk 140 m² k prodeji Na pomezí,Praha 5 - Košíře; 21 679 000 Kč,terasa,výtah,bezbariérový,skeletová stavba,osobní vlastnictví,novostavby.,None,None,None,None,None,None,None,None


In [80]:
data_v3=pd.DataFrame(data_v2.iloc[:,1])

In [81]:
data_v3

,1
0,Praha 5 - Hlubočepy; 4 274 096 Kč (Uvedená ce...
1,Praha 4 - Nusle; 7 990 000 Kč (včetně poplatků
2,Praha 6 - Bubeneč; 8 900 000 Kč
3,Praha 6 - Střešovice; 3 299 000 Kč
4,Praha 4 - Nusle; 9 490 000 Kč
...,...
4699,Praha 5 - Stodůlky; 5 850 000 Kč
4702,9
4704,Praha 3 - Žižkov; 4 290 000 Kč
4705,Praha 5 - Košíře; 21 679 000 Kč


In [85]:
data_v4 = data_v3[1].astype(str).str[:9]

In [88]:
data_v4.head(50)

0      Praha 5 
1      Praha 4 
2      Praha 6 
3      Praha 6 
4      Praha 4 
5      Praha 5 
6      Praha 5 
7      Praha 8 
8      Praha 6 
9      Praha - 
10     Praha 4 
11     Praha - 
12     Praha 4 
14     Praha 4 
15     Praha 4 
16     Praha 4 
17        garáž
18       balkón
19     Praha - 
20     Praha - 
21     Praha 3 
22     Praha 5 
23     Praha 9 
24     Praha 10
25     parkovac
26     Praha - 
27     parkovac
28     Praha 1 
29     Praha 4 
30     Praha - 
31     Praha - 
32     Praha - 
33     Praha - 
34     Praha 10
35     Praha 4 
36     Praha 10
37     Praha 3 
38     Praha 10
39     Praha 10
40     Praha 2 
41     Praha 6 
42     Praha 6 
43     Praha - 
44       terasa
45     Praha 4 
47     Praha - 
48     Praha - 
49     Praha 3 
50     Praha 3 
51     Praha - 
Name: 1, dtype: object

In [98]:
data_v4.value_counts()

 Praha 5     700
 Praha 4     618
 Praha 10    481
 Praha 9     438
 Praha -     376
 Praha 8     312
 Praha 3     298
 Praha 2     260
 Praha 6     254
 Praha 1     241
 Praha 7     118
 balkón       44
 lodžie       27
 Praha; 1     24
 cihlová      22
 výtah        22
 Praha; 5     20
 terasa       20
 parkovac     15
 Praha 4;     14
 včetně p     11
 Praha 6;     10
 Praha 9;      8
 Praha; 7      6
 Praha; 4      6
 Praha; 6      5
 Praha; 3      5
 Praha 8;      4
 Praha; 9      3
 panelová      3
 Praha; 2      3
 garážové      3
 Praha 1;      2
 garáž         2
 Praha; 8      2
 možnost       2
 Praha 5;      1
 právního      1
 včetně k      1
  9            1
 smíšená       1
 B2 9          1
 Praha 2;      1
 + 1% pro      1
 včetně D      1
 bezbarié      1
 na byt s      1
Name: 1, dtype: int64

In [100]:
data_v1 = data_v1[['Title','Type','Area','Latitude','Longitude','Coordinates','Price(CZK)']]

In [102]:
data_v1

,Title,Type,Area,Latitude,Longitude,Coordinates,Price(CZK)
0,"Byt 1+kk 43 m² k prodeji Hugo Haase, Praha 5 -...",1+kk,43,50.0290,14.3622,"(50.029, 14.3623)",4274096
1,"Byt 3+kk 75 m² k prodeji Na Veselí, Praha 4 - ...",3+kk,75,50.0560,14.4414,"(50.056, 14.4414)",7990000
2,"Byt 2+1 71 m² k prodeji Rooseveltova, Praha 6 ...",2+1,71,50.1036,14.3994,"(50.1036, 14.3995)",8900000
3,"Byt 1+kk 27 m² k prodeji Střešovická, Praha 6 ...",1+kk,27,50.0930,14.3773,"(50.0931, 14.3773)",3299000
4,"Byt 4+kk 100 m² k prodeji 5. května, Praha 4 -...",4+kk,100,50.0610,14.4318,"(50.061, 14.4318)",9490000
...,...,...,...,...,...,...,...
4699,"Byt 2+kk 58 m² k prodeji Tlumačovská, Praha 5 ...",2+kk,58,50.0399,14.3357,"(50.0399, 14.3358)",5850000
4702,Byt 4+kk 126 m² k prodeji Praha 5; 9 989 000 K...,4+kk,126,50.0835,14.4341,"(50.0835, 14.4341)",9989000
4704,"Byt 1+kk 35 m² k prodeji Olšanská, Praha 3 - Ž...",1+kk,35,50.0833,14.4662,"(50.0834, 14.4662)",4290000
4705,"Byt 4+kk 140 m² k prodeji Na pomezí, Praha 5 -...",4+kk,140,50.0614,14.3663,"(50.0615, 14.3663)",21679000


In [103]:
data_v1.to_csv('prague_house_prices_v1.csv', index=False)